<a href="https://colab.research.google.com/github/Rad-Monk/Code/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as tfb
import tensorflow_datasets as tfds


In [ ]:
train_data, validation_data, test_data = tfds.load(
    name ='imdb_reviews',
    split = ('train[:60%]', 'train[60%:]', 'test'),
    as_supervised = True
)

By setting as_supervised, the data is loaded in the format of being fed into a supervised model data, basically each input is a tuple of 2 elements, first being the sentence and the other being the label, if we set it to false we will get 2 dictionary keys instead with values of all inputs and labels who=ich would bneed to be extracted later

the split is basically a train vall test split, first part is train, 2nd is test and 3rd is test. very flexible

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [ ]:
train_labels_batch

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = tfb.KerasLayer(
    embedding,
    input_shape=[],
    dtype= 'string',
    trainable = True
)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'relu'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(
    optimizer = 'adam',
    loss= tf.keras.losses.BinaryCrossentropy(from_logits = True),
    metrics = 'accuracy'
)

In [ ]:
epochs = 10
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs = epochs,
                    verbose = 1,
                    validation_data = validation_data.batch(512))

Epoch 1/10
30/30 [==============================] - 49s 2s/step - loss: 0.6942 - accuracy: 0.5004 - val_loss: 0.6932 - val_accuracy: 0.4990
Epoch 2/10
30/30 [==============================] - 60s 2s/step - loss: 0.6931 - accuracy: 0.5007 - val_loss: 0.6931 - val_accuracy: 0.4990
Epoch 3/10
30/30 [==============================] - 55s 2s/step - loss: 0.6931 - accuracy: 0.5007 - val_loss: 0.6931 - val_accuracy: 0.4991
Epoch 4/10
30/30 [==============================] - 57s 2s/step - loss: 0.6930 - accuracy: 0.5008 - val_loss: 0.6931 - val_accuracy: 0.4991
Epoch 5/10
30/30 [==============================] - 47s 2s/step - loss: 0.6928 - accuracy: 0.5009 - val_loss: 0.6929 - val_accuracy: 0.4991
Epoch 6/10
30/30 [==============================] - 49s 2s/step - loss: 0.6709 - accuracy: 0.5663 - val_loss: 0.6166 - val_accuracy: 0.7464
Epoch 7/10
30/30 [==============================] - 43s 1s/step - loss: 0.5574 - accuracy: 0.8367 - val_loss: 0.5387 - val_accuracy: 0.8455
Epoch 8/10
30/30 [==

train_data.shuffle is used to shuffle the data, 10000 at a time, so that model doesn't overfit on order of data, after that we use batch so that the model processes 512 elements at a time.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose = 2)
for name,value in zip(model.metrics_names, results):
  print('%s: %.3f' % (name,value))

49/49 - 10s - loss: 0.5374 - accuracy: 0.8476 - 10s/epoch - 210ms/step
loss: 0.537
accuracy: 0.848


zip makes pair of the items provided